# 커넥션 풀 연습

In [1]:
from mysql import connector
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)
print("설치 완료")

2.2.3
2.2.5
설치 완료


In [7]:
# [1단계] 필요 라이브러리 불러오기
from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import pooling
import pandas as pd

# [2단계] .env 파일 로드
load_dotenv(dotenv_path='.env', override=True)

# [3단계] DB 설정값 환경변수에서 불러오기
db_config = {
    'host': os.getenv('MYSQL_HOST'),
    'port': int(os.getenv('MYSQL_PORT')),
    'user': os.getenv('MYSQL_USER'),
    'password': os.getenv('MYSQL_PASSWORD'),
    'database': os.getenv('MYSQL_DATABASE'),
    'charset': os.getenv('MYSQL_CHARSET')
}

# [4단계] 커넥션 풀 생성 (애플리케이션 전체에서 1회만 실행)
connection_pool = pooling.MySQLConnectionPool(
    pool_name=os.getenv('MYSQL_POOL_NAME'),           # 예: mypool
    pool_size=int(os.getenv('MYSQL_POOL_SIZE')),      # 예: 5
    pool_reset_session=True,                          # 세션 초기화 여부
    **db_config
)

# ──────────────────────────────────────────────────────────────
# [5단계-1] 단순 조회용 (커넥션을 함수 내부에서 꺼냄 - 추천: 단건 조회용)
def fetch_table_as_dataframe(table_name: str) -> pd.DataFrame:
    """
    지정된 테이블을 커넥션 풀에서 연결을 꺼내 조회하여 DataFrame으로 반환 (단건 조회 전용).
    """
    try:
        connection = connection_pool.get_connection()
        cursor = connection.cursor()
        query = f"SELECT * FROM {table_name};"
        cursor.execute(query)

        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(rows, columns=columns)

    except mysql.connector.Error as err:
        print(f"❗ MySQL 에러 발생: {err}")
        return pd.DataFrame()

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals() and connection.is_connected():
            connection.close()

# ──────────────────────────────────────────────────────────────
# [5단계-2] 다중 쿼리/재사용용 (커넥션을 함수 외부에서 주입)
def fetch_table_with_connection(conn, table_name: str) -> pd.DataFrame:
    """
    주어진 커넥션을 사용해 테이블을 조회하여 DataFrame으로 반환 (재사용/트랜잭션용).
    """
    try:
        cursor = conn.cursor()
        query = f"SELECT * FROM {table_name};"
        cursor.execute(query)

        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(rows, columns=columns)

    except mysql.connector.Error as err:
        print(f"❗ MySQL 에러 발생: {err}")
        return pd.DataFrame()

    finally:
        if 'cursor' in locals():
            cursor.close()

# ──────────────────────────────────────────────────────────────
# [6단계] 함수 사용 예시

# ✅ 방법 1: 단건 조회용 (내부에서 커넥션 꺼냄)
df1 = fetch_table_as_dataframe("books")
print("📘 books 테이블 결과:")
print(df1.head())

# ✅ 방법 2: 외부 커넥션 주입 방식 (복수 테이블 조회 시 유리)
conn = connection_pool.get_connection()
df2 = fetch_table_with_connection(conn, "books")
conn.close()  # 마지막에 커넥션 반환

print("\n📘 books 테이블 결과 2:")
print(df2.head())


📘 books 테이블 결과:
   id                                              title          author  \
0   1  Choose Yourself! Be Happy, Make Millions, Live...  James Altucher   
1   2  Skip the Line: The 10,000 Experiments Rule and...  James Altucher   
2   3  The Power of No: Because One Little Word Can B...  James Altucher   
3   4                               The 48 Laws of Power   Robert Greene   
4   5                                            Mastery   Robert Greene   

       genre  release_year  
0  self-help          2013  
1  self-help          2021  
2  self-help          2014  
3  self-help          1998  
4  self-help          2012  

📘 books 테이블 결과 2:
   id                                              title          author  \
0   1  Choose Yourself! Be Happy, Make Millions, Live...  James Altucher   
1   2  Skip the Line: The 10,000 Experiments Rule and...  James Altucher   
2   3  The Power of No: Because One Little Word Can B...  James Altucher   
3   4                         